In [1]:
# Import essential libraries for data handling, visualization, and analysis
import pandas as pd # For data manipulation and analysis
import matplotlib.pyplot as plt # For creating static visualizations
import seaborn as sns # For enhanced statistical data visualization
import numpy as np # For numerical operations

# Import statistical tools
from scipy.stats import spearmanr # For computing Spearman rank correlation
from statsmodels.stats.multitest import multipletests # For multiple testing correction (e.g., FDR)
from statsmodels.formula.api import ols
import statsmodels.api as sm

# Import scikit-learn tool for handling missing values
from sklearn.impute import SimpleImputer # For imputing missing values

# System and warning configuration
import os
import warnings
warnings.filterwarnings('ignore')  # Suppress all warnings
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE" # Prevents conflicts in certain parallel computing environments (e.g., on MacOS or when using MKL)

### Load and Prepare Dataset

In this step, we load the raw behavioral dataset from a .parquet file into a DataFrame. We also standardize gender-related columns (GR_Gender and Gender) by converting all values to lowercase, ensuring consistent formatting for downstream analysis.

In [3]:
# Define the path to the raw data file (Parquet format)
file_path = "raw/ambitus_0_15_log_24_07_2025.parquet"
# load the data into a DataFrame
df = pd.read_parquet(file_path)
# Standardize gender-related columns by converting text to lowercase
df['GR_Gender'] = df['GR_Gender'].str.lower()
df['Gender'] = df['Gender'].str.lower()

### SimpleImputer example

To ensure robust statistical analysis, missing values represented by -1 were first converted to NaN. We then applied SimpleImputer from scikit-learn using the mean imputation strategy on numeric columns that are not fully missing. This ensures each feature can be utilized without being dropped due to incomplete data.

-> Playground cell <-

In [8]:
# Replace placeholder values (-1) with actual NaNs for proper missing value handling
df_SI = df.replace(-1, np.nan)
# Select all numeric columns from the DataFrame
numeric_cols = df_SI.select_dtypes(include="number").columns
# Keep only numeric columns that are not entirely NaN
numeric_cols = numeric_cols[~df_SI[numeric_cols].isna().all()]

# Initialize a SimpleImputer with the mean strategy
# (You can also use strategies like 'median', 'most_frequent', etc.) 
imputer = SimpleImputer(strategy="mean")   # or 'median', 'most_frequent', etc.
# Apply the imputer to fill missing values in numeric columns
imputed_array = imputer.fit_transform(df_SI[numeric_cols])
# Reconstruct a DataFrame from the imputed array with original indices and column names
df_imputed = pd.DataFrame(imputed_array,
                          columns=numeric_cols,
                          index=df_SI.index)
# Update the original DataFrame with the imputed values
df_SI.loc[:, numeric_cols] = df_imputed

In [10]:
display(df_SI)

,Animal,Generation,Season,Separation,G_S,Paradigm,Date_Ambitus,Year,NR,Group,...,Expl_E_BEF_Loco_ratio,Expl_I_BEF_Loco_ratio,Expl_EI_BEF_Loco_ratio,Expl_E_TOT_Loco_ratio,Expl_I_TOT_Loco_ratio,Expl_E_I_TOT_Loco_ratio,Eff_Expl_E,Eff_Expl_I,Eff_Expl_EI,E_E
0,LE2F1_1,0,Autumn,2019-08-26,0_Autumn,1,2019-10-07,0,1,Lisket,...,1.846154,1.076923,2.923077,1.846154,1.076923,2.923077,1.000000,0.75,0.875000,0.8750
1,LE2F2_1,0,Autumn,2019-08-26,0_Autumn,1,2019-10-07,0,1,Lisket,...,1.571429,1.857143,3.428571,1.500000,2.100000,3.600000,1.000000,1.00,1.000000,1.0000
2,LE2F3_1,0,Autumn,2019-08-26,0_Autumn,2,2019-10-07,0,1,Lisket,...,1.000000,1.800000,2.800000,0.769231,1.000000,1.769231,0.689107,1.00,0.533333,1.0000
3,LE2F4_1,0,Autumn,2019-08-26,0_Autumn,2,2019-10-07,0,1,Lisket,...,1.666667,3.166667,4.833333,1.857143,2.357143,4.214286,0.689107,1.00,0.500000,1.0000
4,LE2F1_2,0,Autumn,2019-08-26,0_Autumn,1,2019-10-07,0,2,Lisket,...,1.333333,0.916667,2.250000,1.333333,0.916667,2.250000,1.000000,1.00,1.000000,1.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5363,Rat28,15,Winter,2024-12-30,15_Winter,2,2025-02-18,6,98,LE,...,0.900000,1.600000,2.500000,0.968750,1.343750,2.312500,0.689107,1.00,0.533333,1.0000
5364,Rat11,15,Winter,2024-12-30,15_Winter,1,2025-02-18,6,99,LE,...,0.600000,0.520000,1.120000,0.600000,0.520000,1.120000,0.875000,1.00,0.937500,0.9375
5365,Rat12,15,Winter,2024-12-30,15_Winter,1,2025-02-18,6,99,LE,...,1.200000,1.200000,2.400000,0.846154,0.807692,1.653846,1.000000,1.00,1.000000,1.0000
5366,Rat33,15,Winter,2024-12-30,15_Winter,2,2025-02-18,6,99,LE,...,1.125000,1.750000,2.875000,0.971429,0.800000,1.771429,0.689107,1.00,0.533333,1.0000


### Feature-Wise Missing Data Summary

To quantify the extent of missing data, we calculate the percentage of missing values for each feature across the entire dataset. This summary is exported as feature_missingness.csv, which is intended for inclusion as Supplementary Table S1 in the manuscript. It provides a transparent overview of data quality and completeness.

In [ ]:
# Calculate the percentage of missing values for each column in the full dataset
missing_pct = df.isna().mean().mul(100).sort_values(ascending=False)

# Export the missingness summary to a CSV file (for Supplementary Table S1)
missing_pct.to_csv("results/feature_missingness.csv",
                   index_label="feature",
                   header=["percent_missing"])

In [ ]:
display(missing_pct)

In [ ]:
missing_pct.head(50)

### Visualizing the Top 30 Most Incomplete Features

To identify the least complete features in the dataset, we visualize the top 30 variables with the highest proportion of missing values. The horizontal bar chart provides an intuitive overview of the data quality, highlighting features that may require imputation, exclusion, or careful interpretation. The plot is saved as a high-resolution PDF for inclusion in reports or supplementary materials.

In [ ]:
# Select the 30 features with the highest proportion of missing values
top30 = missing_pct.head(30)
# Create a horizontal bar plot showing the missingness of these top 30 features
plt.figure(figsize=(7, 9))
top30[::-1].plot.barh(color="steelblue") # Plot in reverse order for better readability
# Label the axes and add a title
plt.xlabel("Missing values (%)")
plt.title("Top 30 features by missing rate")
plt.tight_layout()
# Adjust layout and save the figure as a high-resolution PDF
plt.savefig("results/fig_missing_bar.pdf", dpi=300)
# Display the plot
plt.show()
plt.close()

### Heatmap of Missing Data by Generation

To explore whether missingness patterns vary across experimental generations, we computed the percentage of missing values for the top 30 features within each generation. The resulting heatmap highlights generation-specific gaps in the data, which may reflect differences in experimental design, data collection protocols, or recording errors. This visualization can guide decisions about feature selection, imputation strategies, or data quality controls.

In [ ]:
# Create a pivot table: rows = features, columns = generations
# For the top 30 features, compute the percentage of missing values per generation
miss_gen = (
    df.loc[:,list(top30.index)+ ["Generation"]].groupby("Generation")
      .apply(lambda x: x.isna().mean())          # Calculate missing ratio for each feature
      .T                                         # Transpose to make features as rows
      .mul(100)                                  # Convert to percentage
)
# Plot a heatmap of missing values across generations for top 30 features
plt.figure(figsize=(12, 8))
sns.heatmap(
    miss_gen,
    cmap="viridis",
    vmin=0, vmax=100,
    cbar_kws={"label": "Missing (%)"},
    linewidths=0.2
)
# Add labels and formatting
plt.title("Missing-value heatmap by generation(Top 30 features by missing rate)")
plt.xlabel("Generation")
plt.ylabel("Feature")
plt.tight_layout()
# Save the figure as high-resolution PDF
plt.savefig("results/fig_missing_heatmap.pdf", dpi=300)
plt.show()
plt.close()


### Spearman Correlation Among Top Behavioral Features

To examine relationships between key behavioral readouts, we first excluded metadata and categorical variables (e.g., IDs, sex, group labels), then filtered the dataset to retain only numeric features with sufficient variance. From this cleaned subset, we selected the 30 most variable features and computed a Spearman correlation matrix to capture monotonic relationships between them. The heatmap provides insight into potential feature clusters, redundancies, or latent behavioral dimensions.

In [ ]:
# Define a list of keywords related to identifiers and non-behavioral metadata
exclude_keywords = ['id', 'group', 'sex', 'trial', 'year', 'gen', 'rat', 'animal']
# Identify columns to exclude if they contain any of the keywords
exclude_columns = [col for col in df.columns if any(key in col.lower() for key in exclude_keywords)]
# Drop the identified non-behavioral columns from the dataset
df_filtered = df.drop(columns=exclude_columns, errors='ignore')

# Keep only numeric columns
df_numeric = df_filtered.select_dtypes(include='number')

# Drop columns with constant values or near-zero variance
df_numeric = df_numeric.loc[:, df_numeric.std() > 0.01]

# Select top 30 variables by standard deviation
top30_columns = df_numeric.std().sort_values(ascending=False).head(30).index
df_top30 = df_numeric[top30_columns]

# Plot the Spearman correlation matrix for the top 30 features
plt.figure(figsize=(8, 6))
sns.heatmap(df_top30.corr(method="spearman"), cmap="coolwarm", center=0, annot=False, fmt=".2f", linewidths=1)
plt.title("Correlation Matrix – Top 30 Behavioral Features", fontsize=16)
# Display the figure
plt.show()

### Generational Drift Analysis

To identify behavioral features that change systematically across generations, we computed Spearman correlations between each numeric feature and the generation number. This non-parametric approach captures monotonic trends regardless of linearity. After excluding non-numeric columns, we applied False Discovery Rate (FDR) correction to adjust for multiple hypothesis testing. The resulting ranked list of features (saved as gen_drift_spearman.csv) highlights traits that may reflect generational drift, such as adaptation, learned behavior, or methodological differences.


In [ ]:
# separate metadata & numeric fesature matrix
meta_cols = ["RatID", "Generation"]                    # Add more metadata columns if needed
all_feat_cols = [c for c in df.columns if c not in meta_cols]

# Select only numeric and boolean columns (booleans are treated as numeric)
numeric_feat_cols = df[all_feat_cols].select_dtypes(
    include=["number", "boolean"]            # bool is OK
).columns.tolist()

# Optionally: print a few of the excluded (non-numeric) columns
dropped = set(all_feat_cols) - set(numeric_feat_cols)
if dropped:
    print(f"{len(dropped)} non-numeric columns skipped: {sorted(dropped)[:5]} ...")

# Compute Spearman correlation between Generation and each numeric feature
results = []
gen = df["Generation"].astype("float") 
for f in numeric_feat_cols:
    rho, p = spearmanr(gen, df[f], nan_policy="omit")
    results.append({"feature": f, "rho": rho, "p": p})
# Compile results and sort by absolute correlation
res = pd.DataFrame(results).sort_values("rho", key=np.abs, ascending=False)
# Apply FDR correction (Benjamini–Hochberg) for multiple testing
res["q"] = multipletests(res["p"], method="fdr_bh")[1]
# Save results to CSV
res.to_csv("results/gen_drift_spearman.csv", index=False)

In [ ]:
display(res)

### Generational Drift – Correlation Heatmap of Top 25 Features

To further investigate the structure of generational drift, we selected the top 25 features with the strongest correlations to generation. By aggregating feature values at the generation level (using medians), we reduced individual variability and highlighted population-level trends. A Spearman correlation heatmap reveals how these features co-vary across generations, suggesting potential latent dimensions or coordinated behavioral shifts over time.

In [ ]:
# Select the top 25 features showing the strongest generational drift
top25 = res.head(25)["feature"]
# Compute the Spearman correlation matrix between these top 25 features
# using their median values per generation (to smooth out noise)
corr_sub = (df.groupby("Generation")[top25]
              .median()
              .corr(method="spearman")          # feature × feature, quick normalisation
              .loc[top25, top25])

# Plot the absolute Spearman correlations as a heatmap
plt.figure(figsize=(8, 10))
sns.heatmap(np.abs(corr_sub), cmap="crest", vmin=0, vmax=1,
            cbar_kws={"label": "abs(Spearman ρ)"})
# Add title and formatting
plt.title("Generational drift – top 25 features")
plt.tight_layout()
# Save the figure as a high-resolution PDF
plt.savefig("results/fig_drift_heatmap.pdf", dpi=300)
plt.show()
plt.close()

### Generational Trends of Key Behavioral Metrics

To illustrate how specific behavioral traits evolve across generations, we plotted three representative metrics using a dual-axis line chart. The left y-axis shows latency (in milliseconds), while the right y-axis displays count-based and ratio-based metrics. Median values and interquartile ranges (25th to 75th percentiles) are visualized to highlight both central tendency and variability. This plot captures long-term trends that may reflect generational adaptation or procedural shifts in the experimental setup.

In [ ]:
# Create a dual-axis plot to visualize multiple metrics with different scales
fig, ax1 = plt.subplots(figsize=(8, 4.5))
ax2 = ax1.twinx() # Secondary y-axis

# Assign consistent colors using Matplotlib's default color cycle
col_map = dict(zip(features_to_plot, ["C0", "C1", "C2"]))

# LEFT y-axis: latency feature (typically larger values)
f_left = "Expl_E_AFT_T"
ax1.plot(df_line["Generation"], df_line[f"{f_left}_med"],
         label=f_left, color=col_map[f_left])
ax1.fill_between(df_line["Generation"],
                 df_line[f"{f_left}_q25"],
                 df_line[f"{f_left}_q75"],
                 alpha=0.20, color=col_map[f_left])

ax1.set_ylabel("Latency (ms)")
ax1.set_xlabel("Generation")

# RIGHT y-axis: count and ratio features (typically smaller scale)
for f in ["Expl_E_AFT_Nr", "Expl_E_TOT_Loco_ratio"]:
    ax2.plot(df_line["Generation"], df_line[f"{f}_med"],
             label=f, color=col_map[f])
    ax2.fill_between(df_line["Generation"],
                     df_line[f"{f}_q25"],
                     df_line[f"{f}_q75"],
                     alpha=0.20, color=col_map[f])

ax2.set_ylabel("Count / Ratio")

# Combine legends from both axes
handles, labels = [], []
for ax in (ax1, ax2):
    h, l = ax.get_legend_handles_labels()
    handles.extend(h)
    labels.extend(l)

ax1.legend(handles, labels, frameon=False, loc="upper left")
# Add plot title and finalize layout
ax1.set_title("Generational trend of key behavioural metrics")
plt.tight_layout()
plt.savefig("results/fig_drift_lineplot.pdf", dpi=300)
plt.show()
plt.close()


### Factorial ANOVA – Group × Sex × Year Effects

To evaluate whether behavioral outcomes differ across experimental groups, sexes, and years, we performed a factorial ANOVA on six key behavioral features. The model includes:
- Main effects: Group, Sex, Year
- Two-way interactions: Group × Year and Sex × Year
  
This analysis identifies which behavioral measures show statistically significant differences across subpopulations or temporal contexts. The results are compiled into a unified ANOVA table (final_anova_df) for further inspection and visualization.

In [ ]:

# Drop rows where grouping variables are missing
df = df.dropna(subset=['GR_Gender', 'Year'])
# Normalize gender information (lowercase for consistency)
df['Sex'] = df['Gender'].str.lower()
df['Group_Sex'] = df['GR_Gender'].str.lower()

# Define variables to test and corresponding feature names in the dataset
anova_targets = {
    'LOCO_TOT': 'Locomotion (Loco_TOT)',
    'LOCO_BEF': 'Locomotion frequency (LOCO_BEF)',
    'EXPL_TOT': 'Exploration (Expl_TOT)',
    'Expl_E_I_BEF_Nr': 'Exploration frequency (Expl_BEF)',
    'L_C': 'Learning capacity (L_C)',
    'E_E': 'Effective exploration ratio (E_E)'
}

# Initialize a list to collect ANOVA results
anova_results = []

# Perform two-way factorial ANOVA for each target variable
# Including main effects and interactions with Year
for feature, description in anova_targets.items():
    formula = f"{feature} ~ C(Group) + C(Sex) + C(Year) + C(Group):C(Year) + C(Sex):C(Year)"
    model = ols(formula, data=df).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    # Annotate results with feature description and variable name
    anova_table["Feature"] = description
    anova_table["Variable"] = anova_table.index
    anova_results.append(anova_table.reset_index(drop=True))

# Combine results into a single DataFram
final_anova_df = pd.concat(anova_results, ignore_index=True)

### ANOVA Summary Table (Formatted for Readability)

The raw factorial ANOVA results are reformatted into a clean summary table. Only the main effects and relevant two-way interactions are retained:
- Gr: Experimental Group
- Sex: Biological Sex
- Year: Testing Year
- Gr/Y: Group × Year interaction
- Sex/Y: Sex × Year interaction

Each cell shows the F-statistic along with the associated p-value, helping to quickly identify which variables are significantly influenced by group, sex, year, or their interactions. This format is well-suited for publication or supplementary tables.

In [ ]:
# Create a copy of the original ANOVA results to preserve the source
df_anova = final_anova_df.copy()

# Define a mapping from raw ANOVA variable terms to simplified labels
label_map = {
    'C(Group)': 'Gr',
    'C(Sex)': 'Sex',
    'C(Year)': 'Year',
    'C(Group):C(Year)': 'Gr/Y',
    'C(Sex):C(Year)': 'Sex/Y'
}

# Filter the DataFrame to retain only the terms of interest
df_anova = df_anova[df_anova['Variable'].isin(label_map.keys())]

# Create a new column for simplified effect labels
df_anova['Effect'] = df_anova['Variable'].map(label_map)

# Format the F and p-values into a readable string
df_anova['F(p)'] = df_anova.apply(
    lambda row: f"{row['F']:.2f} (p < {row['PR(>F)']:.4f})", axis=1
)

# Pivot the table so each effect is a column and each feature is a row
table_formatted = df_anova.pivot(index='Feature', columns='Effect', values='F(p)')

# Reorder columns to match desired output
ordered_cols = ['Gr', 'Sex', 'Year', 'Gr/Y', 'Sex/Y']
table_formatted = table_formatted.reindex(columns=ordered_cols)
display(table_formatted)